In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/age_gender.csv")

In [ ]:
df.ethnicity.value_counts()

0    10078
1     4526
3     3975
2     3434
4     1692
Name: ethnicity, dtype: int64

In [ ]:
df

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...
3,1,2,0,20161220144911423.jpg.chip.jpg,193 197 198 200 199 200 202 203 204 205 208 21...
4,1,2,0,20161220144914327.jpg.chip.jpg,202 205 209 210 209 209 210 211 212 214 218 21...
...,...,...,...,...,...
23700,99,0,1,20170120221920654.jpg.chip.jpg,127 100 94 81 77 77 74 99 102 98 128 145 160 1...
23701,99,1,1,20170120134639935.jpg.chip.jpg,23 28 32 35 42 47 68 85 98 103 113 117 130 129...
23702,99,2,1,20170110182418864.jpg.chip.jpg,59 50 37 40 34 19 30 101 156 170 177 184 187 1...
23703,99,2,1,20170117195405372.jpg.chip.jpg,45 108 120 156 206 197 140 180 191 199 204 207...


In [ ]:
df["age"] = pd.cut(df["age"],bins=[0,3,18,45,64,116],labels=["0","1","2","3","4"])

In [ ]:
df.head()

,age,ethnicity,gender,img_name,pixels
0,0,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...
1,0,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...
2,0,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...
3,0,2,0,20161220144911423.jpg.chip.jpg,193 197 198 200 199 200 202 203 204 205 208 21...
4,0,2,0,20161220144914327.jpg.chip.jpg,202 205 209 210 209 209 210 211 212 214 218 21...


In [ ]:
len(df['age'].unique())

5

In [ ]:
x = df['pixels']
y = df[['age','ethnicity','gender','age']]

In [ ]:
new_x = []
for i, row in df.iterrows():
    l = row['pixels'].split(" ")
    new_x.append(l)

In [ ]:
new_x =np.array(new_x)

In [ ]:
new_x.shape

(23705, 2304)

In [ ]:
new_x

array([['129', '128', '128', ..., '146', '146', '146'],
       ['164', '74', '111', ..., '182', '170', '148'],
       ['67', '70', '71', ..., '112', '111', '108'],
       ...,
       ['59', '50', '37', ..., '98', '78', '78'],
       ['45', '108', '120', ..., '32', '35', '35'],
       ['156', '161', '160', ..., '190', '184', '174']], dtype='<U3')

In [ ]:
new_x=new_x.reshape(new_x.shape[0],48,48,1)

In [ ]:
new_x.shape

(23705, 48, 48, 1)

In [ ]:
new_x = np.array(new_x,'float64')

In [ ]:
new_x = new_x / 255.0
new_x = new_x / 255.0

### Creation a model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
import tensorflow as tf

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, InputLayer
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy

In [ ]:
early_stopping = EarlyStopping(patience=10, 
                               min_delta=0.001,
                               restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                           patience = 2,
                                           verbose=1,
                                           factor=0.5,
                                           min_lr = 0.00001)

In [ ]:
gender=tf.keras.utils.to_categorical(y['gender'],num_classes=2)
ethnicity = tf.keras.utils.to_categorical(y['ethnicity'],num_classes=5)
age = tf.keras.utils.to_categorical(df['age'],num_classes=5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_ethnicity, X_test_ethnicity, y_train_ethnicity, y_test_ethnicity = train_test_split(new_x, ethnicity, test_size= 0.3)
X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(new_x, gender, test_size= 0.3)
X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(new_x, age, test_size=0.3, random_state=42)
# X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(x, age, test_size= 0.3)

In [ ]:
def my_model(num_classes, activation, loss):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding = "same", input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, kernel_size=(3,3),activation="relu",padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, kernel_size=(3,3),activation="relu",padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256,activation="relu"))
    model.add(Dense(num_classes, activation=activation))
    
    model.compile(optimizer='Adam',
              loss= loss,
              metrics=['accuracy'])
    return model

In [ ]:
epochs = 200  # for better result increase the epochs
batch_size = 64
model_ethnicity = my_model(5,"softmax",'categorical_crossentropy')
history_ethnicity = model_ethnicity.fit(X_train_ethnicity, y_train_ethnicity, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_test_ethnicity,y_test_ethnicity), steps_per_epoch= X_train_ethnicity.shape[0] // batch_size,callbacks= [early_stopping, learning_rate_reduction])

Epoch 1/200


ValueError: ignored

In [ ]:
epochs = 200  # for better result increase the epochs
batch_size = 64
loss, acc = model_ethnicity.evaluate(X_test_ethnicity, y_test_ethnicity, verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))
model_ethnicity.save("model.h5")

KeyboardInterrupt: ignored

In [ ]:
epochs = 200  # for better result increase the epochs
batch_size = 64
model_gender = my_model(2, "sigmoid", "binary_crossentropy")
history_gender = model_gender.fit(X_train_gender, y_train_gender, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_test_gender,y_test_gender), steps_per_epoch= X_train_gender.shape[0] // batch_size, callbacks= [early_stopping, learning_rate_reduction])

Epoch 1/200
259/259 [==============================] - 107s 411ms/step - loss: 0.4845 - accuracy: 0.7736 - val_loss: 0.9014 - val_accuracy: 0.4850 - lr: 0.0010
Epoch 2/200
259/259 [==============================] - 99s 384ms/step - loss: 0.3511 - accuracy: 0.8399 - val_loss: 0.7331 - val_accuracy: 0.4989 - lr: 0.0010
Epoch 3/200
259/259 [==============================] - 100s 387ms/step - loss: 0.3177 - accuracy: 0.8586 - val_loss: 0.5971 - val_accuracy: 0.6384 - lr: 0.0010
Epoch 4/200
259/259 [==============================] - 99s 381ms/step - loss: 0.2932 - accuracy: 0.8684 - val_loss: 1.1226 - val_accuracy: 0.5332 - lr: 0.0010
Epoch 5/200
259/259 [==============================] - 99s 383ms/step - loss: 0.2666 - accuracy: 0.8821 - val_loss: 0.8699 - val_accuracy: 0.5150 - lr: 0.0010
Epoch 6/200
259/259 [==============================] - 99s 381ms/step - loss: 0.2472 - accuracy: 0.8941 - val_loss: 0.9098 - val_accuracy: 0.5389 - lr: 0.0010
Epoch 7/200
259/259 [=======================

In [ ]:
loss, acc = modtel_gender.evaluate(X_test_gender,y_test_gender, verbose=0)

In [ ]:
acc

0.7529527544975281

In [ ]:
model_gender.save("model_gender.h5")

In [ ]:
epochs = 200  # for better result increase the epochs
batch_size = 10
model_age = my_model(5,"softmax",'categorical_crossentropy')
history_age = model_age.fit(X_train_age, y_train_age, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_test_age,y_test_age), 
                            steps_per_epoch= X_train_age.shape[0] // batch_size, 
                            callbacks= [early_stopping, 
                            learning_rate_reduction])

Epoch 1/200
1659/1659 [==============================] - 143s 85ms/step - loss: 0.8867 - accuracy: 0.6657 - val_loss: 0.7267 - val_accuracy: 0.7129 - lr: 0.0010
Epoch 2/200
1659/1659 [==============================] - 134s 81ms/step - loss: 0.7182 - accuracy: 0.7203 - val_loss: 2.1699 - val_accuracy: 0.1237 - lr: 0.0010
Epoch 3/200
1659/1659 [==============================] - 132s 80ms/step - loss: 0.6624 - accuracy: 0.7404 - val_loss: 1.7768 - val_accuracy: 0.1666 - lr: 0.0010
Epoch 4/200
1659/1659 [==============================] - 133s 80ms/step - loss: 0.6228 - accuracy: 0.7547 - val_loss: 2.5801 - val_accuracy: 0.1112 - lr: 0.0010
Epoch 5/200
1659/1659 [==============================] - 134s 81ms/step - loss: 0.5958 - accuracy: 0.7637 - val_loss: 6.3783 - val_accuracy: 0.4558 - lr: 0.0010
Epoch 6/200
1659/1659 [==============================] - 134s 81ms/step - loss: 0.5673 - accuracy: 0.7723 - val_loss: 7.8963 - val_accuracy: 0.5783 - lr: 0.0010
Epoch 7/200
1659/1659 [===========

In [ ]:
loss, acc = model_age.evaluate(X_test_age,y_test_age, verbose=0)

In [ ]:
acc

0.7128796577453613

In [ ]:
model_age.save("model_age.h5")